### 树结构的术语

- 节点 node：保存数值
    - 子节点 children：入边都来自同一节点的节点，被称为这个节点的子节点
    - 父节点 parent：一个节点是所有其出边节点的父节点
    - 兄弟节点 sibling：拥有同一个父节点的节点，互为兄弟节点
    - 叶节点 leaf：没有子节点的节点
- 边 edge：连接两个节点，表示节点之间的关系，有出入方向；每个节点可以有多个边
- 根 root：树中唯一没有入边的节点
- 路径 path：由边连接起来的节点的列表
- 子树 subtree：一个节点和其所有子孙节点，构成一个子树
- 层级 level：从根到这个节点经过的边的数量，就称为这个节点的层级
- 高度 height：一棵树所有节点的最大层级，被称为这棵树的高度
- 二叉树 binary tree：每个节点最多只有2个子节点的树

**用嵌套列表来实现二叉树**
```
[root, left, right]
```
BinaryTree来创建一颗二叉树

insertRight/insertLeft 将节点插入树

get/setRootVal 获取或设置根节点

getLeft/RightChild 来获取左/右子树


In [3]:
def BinaryTree(r):
    return [r, [], []]

def insertLeft(root, newBranch):
    t = root.pop(1)
    if len(t) > 1:
        root.insert(1, [newBranch, t, []])
    else:
        root.insert(1, [newBranch, [], []])
    return root

def insertRight(root, newBranch):
    t = root.pop(2)
    if len(t) > 1:
        root.insert(2, [newBranch, [], t])
    else:
        root.insert(2, [newBranch, [], []])
    return root

def getRootVal(root):
    return root[0]

def setRootVal(root, newVal):
    root[0] = newVal

def getLeftChild(root):
    return root[1]

def getRightChild(root):
    return root[2]

In [5]:
#test
r = BinaryTree(3)
insertLeft(r, 4)
insertLeft(r, 5)
insertRight(r, 6)
insertRight(r, 7)
l = getLeftChild(r)
print(l)

setRootVal(l, 9)
print(r)
insertLeft(l, 11)
print(r)
print(getRightChild(getRightChild(r)))

[5, [4, [], []], []]
[3, [9, [4, [], []], []], [7, [], [6, [], []]]]
[3, [9, [11, [4, [], []], []], []], [7, [], [6, [], []]]]
[6, [], []]


**用链表实现二叉树**

每个节点保存数据和指向左右子节点的链接

In [11]:
class BinaryTree2:
    def __init__(self, rootObj):
        self.key = rootObj
        self.leftChild = None
        self.rightChild = None

    def insertLeft(self, newNode):
        if self.leftChild == None:
            self.leftChild = BinaryTree(newNode)
        else:
            t = BinaryTree(newNode)
            t.leftChild = self.leftChild
            self.leftChild = t

    def insertRight(self, newNode):
        if self.rightChild == None:
            self.rightChild = BinaryTree(newNode)
        else:
            t = BinaryTree(newNode)
            t.rightChild = self.rightChild
            self.rightChild = t

    def setRootVal(self, val):
        self.key = val

    def getRootVal(self):
        return self.key

    def getLeftChild(self):
        return self.leftChild

    def getRightChild(self):
        return self.rightChild

### 树的应用

1. 解析树（语法树）
2. 表达式解析：叶节点保存操作数，内部节点保存操作符

表达式解析的建立：
- 将公式拆解成token
- 创建空树，当前节点为根节点
- 读取到左括号，就创建左子节点，当前节点下降到该节点
- 读取到操作数，就将该节点设置为该操作数，当前节点上升到父节点
- 读取到操作符，当前节点设置为操作符，创建右节点，当前节点下降
- 读取到另一个操作数，当前节点设置为该操作数，上升到父节点
- 读取到右括号，上升到父节点
- 目前二叉树的方法可以设置节点的数据和下降到子节点，但是没有专门的上升到父节点的方法，因此可以用一个栈来记录跟踪父节点


In [9]:
#借用之前的stack类来辅助实现
class Stack:
    def __init__(self):
        self.items = []

    def isEmpty(self):
        return self.items == []

    def push(self, item):
        self.items.append(item)

    def pop(self):
        return self.items.pop()

    def peek(self):
        return self.items[len(self.items) - 1]

    def size(self):
        return len(self.items)

In [12]:
#code
def buildParseTree(fpexp):
    fplist = fpexp.split()
    pStack = Stack()
    eTree = BinaryTree2('')
    pStack.push(eTree)
    currentTree = eTree
    for i in fplist:
        if i == '(':
            currentTree.insertLeft('')
            pStack.push(currentTree) #下降前将节点推入栈
            currentTree = currentTree.getLeftChild() #将当前节点设为子节点
        elif i not in ['+', '-', '*', '/', ')']:
            currentTree.setRootVal(int(i))
            parent = pStack.pop() #弹出父节点
            currentTree = parent
        elif i in ['+', '-', '*', '/']:
            currentTree.setRootVal(i)
            currentTree.insertRight('')
            pStack.push(currentTree)
            currentTree = currentTree.getRightChild()
        elif i == ')':
            currentTree = pStack.pop() #出栈上升
        else:
            raise ValueError
    return eTree

** 利用表达式解析树求值**

可以用递归算法来处理

In [13]:
#这里使用operator模块来简化求值部分的代码
import operator
def evaluate(parseTree):
    opers = {'+': operator.add, '-': operator.sub, '*': operator.mul, '/': operator.truediv}

    leftC = parseTree.getLeftChild()
    rightC = parseTree.getRightChild()

    if leftC and rightC:
        fn = opers[parseTree.getRootVal()]
        return fn(evaluate(leftC), evaluate(rightC))
    else:
        return parseTree.getRootVal()

In [16]:
#test
gs = '(  5 * ( 3 + 4 ) )'
pt = buildParseTree(gs)
e = evaluate(pt)

In [21]:
print(pt.getRootVal())
l = pt.getLeftChild()
print(l.getRootVal())
r = pt.getRightChild()
print(r.getRootVal())
print(r.getRightChild().getRootVal())
print(e)

*
5
+
4
35
